# Computer Vision

<div class="alert alert-block alert-info"> <strong> </strong> [ Write your surname.name here (like that surname.name) ]  
</div>

#### Welcome to this exercise which introduces Computer Vision!

<img style="float:right;" src="./img/pinhole_model.png" width= 50% />

<br>
While photogrammetry has over 100 years of tradition, the past several years has seen Computer Vision applied in the geospatial domain.
<br> <br>


Newer technologies often come with their own jargon (terms, definitions and language). 

This exercise is not meant to be exhaustive but serves as a gentle introduction to the concepts and potential of Computer Vision.

<div class="alert alert-block alert-info"> <strong> Lets first define the core difference between these two methods. </strong>  <br> <br>

**Computer Vision**: interpretation and understanding of visual data often involving real-time decision-making by machines. Enable machines to interpret and understand visual data. Medical imaging and autonomous navigation|<br><br>
**Photogrammetry**: reconstruct 3D structures and spatial relationships. Accurately measure and reconstruct the three-dimensional shape. Remote sensing and mapping.   
</div>

**In order for realise the goal of Computer Vision it is often necessary to harvest 3D data from imagery**. While this is also the aim of photogrammetry; the processes differ slightly.

|Aspect|Traditional Photogrammetry|Computer Vision|
|---|---|---|
|Stereo|Primarily relies on **_stereo matching_** to create 3D models|Combines stereo vision with **_advanced techniques_** like Structure from Motion Multi-view Stereo (SfM-MVS) and SLAM (Simultaneous Localization and Mapping)|
|Calibration|**_Precise sensor calibration_** for accurate results|May involve more **_automated_** calibration procedures|
|Processing Speed|Time-consuming manual processes|Often performs real-time or **_near-real-time processing_** thanks to powerful hardware and **_optimized algorithms_**.|
|Data Volume|Deals with smaller datasets of images.|Analyzes large datasets of images, videos, and 3D point clouds.|
|Data Availability|Data acquisition and processing may be costly and time-consuming.|Access to data is becoming more abundant and affordable, thanks to the proliferation of digital cameras and sensors|
|Accuracy|High accuracy _(or rather the **confidence** in the quality)_ making it suitable for geospatial applications|Accuracy varies depending on the application and the quality of data and algorithms|


<div class="alert alert-danger">
  <strong>REQUIRED!</strong> 
  
You are required to insert your outputs and any comment into this document. 
    
The **aim** of this exercise is for you to execute this Notebook on **_a series of photographs you have captured yourself_**. The document you submit should therefore contain the existing text in addition to:   
        

 - Plots and other outputs from exec the code cells  
 - Discussion of your plots and other outputs as well as conclusions reached.  
 - This should also include any hypotheses and assumptions made as well as factors that may affect your conclusions.
</div>

In [ ]:
#- load the magic
import cv2
import numpy as np

In [3]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import os
dir = os.path.join("drive/My Drive/UCTJuly-Nov2023/APG3012S-RSandP/assignments/collinearity/data/sfm/")#data/sfm")

<div class="alert alert-block alert-success"> <strong> For this exercise we will harvest 3D data from imagery with Computer Vision. </strong>  
<br><br>
We will do so in two stages. 
<br><br>  
    
&nbsp;&nbsp;&nbsp;&nbsp;**1. Traditional stereo-vision** _(two photographs)_  
&nbsp;&nbsp;&nbsp;&nbsp;**2. Structure-from-Motion (SfM)** _(with several images to compare with traditional photogrammetry)_
<!-- &nbsp;&nbsp;&nbsp;&nbsp;**2.** extend the SfM process with **Multi-view Stereo (MVS)** _(and show how the method accomodates many images)_-->

In the process we will work through such concepts as Feature Matching, Essential and Fundamental matrices and Disparity
</div>

In [30]:
#- path
input_dir = os.path.join(dir, 'stereo')

### 1. Stereo-vision

Before we _really_ get into **Computer Vision**; it might be helpful to introduce modern _'algorithm-based'_ stereo photogrammetry. Modern **Computer Vision** is infact already very mature with completely automated workflows that harvest 3D information via Deep Learning _(neural networks)_. 

We therefore take a step back and start with the basics to understand how these procedures execute a solution. Two images, of the same feature, taken from two different viewpoints.

<table><tr>
<td> <img src="./data/stereo/im0.png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="./data/stereo/im0.png" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>

#### 1. a. Feature Matching

In order to harvest 3D data we need the location and orientation of the cameras; at the moment the image was captured. 

Like traditional **Photogrammerty** which needs _**some**_ variables _(ground control, location and orientation of photographs)_ to solve for the **collinearity equation**; **Computer Vision** also needs a few known variables. Even so; modern **Computer Vision** can recover the geometry and pose of the camera _(the relative orientation)_ from the imagery itself. It can do so through feature detection and matching algorithms which work in two steps:

&nbsp;&nbsp;&nbsp;&nbsp;i) find features _(known as **keypoints**)_ in images and  
&nbsp;&nbsp;&nbsp;&nbsp;ii) **match** corresponding features _(connect the features that match and disgard those that don't)_

In [50]:
#- feature matching
def PointMatchingAKAZE(img1, img2, path):

    akaze = cv2.AKAZE_create()
    kp1, des1 = akaze.detectAndCompute(img1, None)
    kp2, des2 = akaze.detectAndCompute(img2, None)

    #- match features
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.knnMatch(des1, des2, k=2)

    #- ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.70 * n.distance:
            good.append(m)

    pts1 = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

    #- cv2.drawMatchesKnn expects list of lists as matches.
    #img3 = cv2.drawMatches(img1, kp1, img2, kp2, good[:50], None, flags=2)
    img3 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, flags=2)
    cv2.imwrite(os.path.join(path, 'imL_x_imR.jpg'), img3)

    return pts1, pts2, good

**keypoints**

<img src="./data/stereo/keypoints.jpg" alt="Drawing" style="width: 1000px;"/> </td>

**matches**

<img src="./data/stereo/im2_x_im4.jpg" alt="Drawing" style="width: 1000px;"/> </td>

<div class="alert alert-block alert-warning"><b>TASK! </b>  </div>

- **You are expected to insert the result** _(the `keypoints` and `..._x_....jpg`)_ **of your own dataset in the `cells`** _(double-click the image and change the file)_ **above and discuss the result.** 

<div class="alert alert-block alert-info"><b>HINT!</b> Open the image in another application. Zoom and look at the number of features and the respective matches. </div> 

{ click in this cell and write your answer here }

In [43]:
#- do (fix) this
def PointMatchingSURF(img1, img2, save = False, path = None):
    #Match with SURF
    surf = cv2.xfeatures2d.SURF_create()
    keypoints1, descriptors1 = surf.detectAndCompute(img1, None)
    keypoints2, descriptors2 = surf.detectAndCompute(img2, None)

    img4 = cv2.drawKeypoints(img1, keypoints1, None)

    cv2.imwrite("KP.jpg", img4)
    bf = cv2.BFMatcher_create()

    matches = bf.match(descriptors1, descriptors2)

    img3 = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches[:200], None, flags=2)

    if save:
        cv2.imwrite('03' + "_x_" + '04' +".jpg", img3)

    return keypoints1, keypoints2,matches

<div class="alert alert-block alert-warning"><b>QUESTION! </b>  </div>

- **This exercise does not use `Scale-invariant feature transform (SIFT)` to match features but executes the `AKAZE` algorithm** _(you are expected to change the `code` and test both solutions)_.
  
    **Discuss the difference between these two methods. Mention their strenghts and weaknessess. A note about intellectual property is expected. Your answer cannot be more than 150 words.**

{ click in this cell and write your answer here }

#### 1. b. Camera calibration

**While we can calibrate a camera with `opencv` we use existing calibration information.** 

You will need to create your own for the **TASK** at the end of this exercise so lets see what it contains.  The `intrinsic.txt` is structured:

\begin{equation}
\begin{pmatrix}
  fx       & 0   & cx   \\
  0       & fy   & cy   \\
  0       & 0   & 1     \\
\end{pmatrix}
\end{equation}

where `fx = f * resX (the number of columns) / sensorSizeX` and `fy = f * resY (the number of rows) / sensorSizeY` with `f = focal length`. 
`cx` and `cy` represent the principle point of the image and, with modern digital cameras, can be calculated with `cx = resX (the number of columns) / 2` and `cy = resY (the number of rows) / 2`.

In [32]:
#- fix this
def findCalibrationMat():
    #with open(input_dir+'intrinsic.txt') as f: #os.path.join(input_dir, 'im3.jpg'))
    with open(os.path.join(dir, 'intrinsic.txt')) as f:
        lines = f.readlines()
    return np.array(
        [l.strip().split(' ') for l in lines],
        dtype=np.float32
    )

#### 1. c. Fundamental and Essential matrices _(relative orientation)_


In **Photogrammetry** we locate and orient _**one**_ image and are able to calculate a 3D coordinate via the **collinearity equation**. When we do intersect **image rays** from more than one image; we do so from already oriented imagery. 

In order to harvest 3D information; we need to know the relationship between the images (_i.e.: the relative orientation)_.

In Computer Vision we call this the **essential and fundamental matrices**. These matrices solve for the geometry and pose of the cameras at the moment the images were captured.

The previous step (feature matching) in **Computer Vision** thus serves two purposes. We recover the _relative orientation_ of all images simultaneously (essential matrix) and describe the relationship between images in the same scene. And use these to map points of one image to lines in another (fundamental matrix). 

Think of an **essential and fundamental matrices** as generalizations of the **collinearity equations** that account for the relative geometry between multiple cameras. They allow for the reconstruction of 3D structures by considering the relationship between corresponding points in multiple cameras.

In [14]:
def FindEssentialMat(kp1, kp2, matches, K):
    
    #- essential matrix
    E, mask = cv2.findEssentialMat(kp1, kp2, K, method = cv2.RANSAC, prob = 0.999, threshold = 0.4, mask = None)
    kp1 = kp1[mask.ravel() ==1]
    kp2 = kp2[mask.ravel() ==1]
    #- fundamental matrix
    #fundamental_matrix, inliers = cv.findFundamentalMat(kp1, kp2, cv.FM_RANSAC)
    
    #- essential matrix to rotation and translation
    _, R, t, mask = cv2.recoverPose(E, kp1, kp2, K)
    
    return E, R, t

<div class="alert alert-block alert-warning"><b>EXTRA!</b> </div> 

- **While beyond the scope of this exercise; the overal aim of the Fundamental and Essential matrices is to recover the Epipolar Geometry and the Epipolar Line. These _narrow_ the search space to recovery the Disparity** _(the horizontal difference)_ **between pixels.**

  **You are welcome to write 100 words on Epipolar Geometry and Epipolar Line and what it does. The choice is yours. Its up to you**

{ click in this cell and write your answer here }

#### 1. d. Disparity


<img style="float:right;" src="./img/Stereo-Disparity-in-Cameras.png" width= 40% />

<br>

We see **Disparity** and _perceive depth_. 

Choose an object 5-meters away from you. Close your left eye... open your left eye. Now close you right eye. Alternate. The object will appear to move _(horizontally)_. Left and right. 

We see _**horizontal displacement**_ and interpret the phenomenon as _3D vision_.

Modern **Computer Vision** harvests 3D information in _exactly the same_ way as the human eye sees. 

Because we have correctly oriented photographs we know the _perceived shift in location_ of any feature. We use this knowledge to create a depth _(disparity)_ map that decribes the distance from the camera.

In [34]:
def rectifyToDisparity(imgL, imgR, K, D, R, t, path):
    
    R1, R2, P1, P2, Q, a, b = cv2.stereoRectify(K, D, K, D, (2964, 2000), R, t)
    map1, map2 = cv2.initUndistortRectifyMap(K, D, R1, P1, (2964, 2000), cv2.CV_16SC2)
    imgLrec = cv2.remap(imgL, map1, map2, cv2.INTER_CUBIC)
    #cv2.imwrite(os.path.join(path, 'imgLrectfy.jpg'), imgLrec)
    
    map3, map4 = cv2.initUndistortRectifyMap(K, D, R2, P2, (2964, 2000), cv2.CV_16SC2)
    imgRrec = cv2.remap(imgR, map3, map4, cv2.INTER_CUBIC)
    #cv2.imwrite(os.path.join(path, 'imgRrectfy.jpg'), imgRrec)
    
    max_disparity = 199
    min_disparity = 23
    num_disparities = max_disparity - min_disparity
    window_size = 5
    stereo = cv2.StereoSGBM_create(minDisparity = min_disparity, numDisparities = num_disparities, blockSize = 5,
                                 uniquenessRatio = 5, speckleWindowSize = 5, speckleRange = 5, disp12MaxDiff = 2,
                                 P1 = 8*3*window_size**2, P2 = 32*3*window_size**2)
    
    stereo2 = cv2.ximgproc.createRightMatcher(stereo)
    
    lamb = 8000
    sig = 1.5
    visual_multiplier = 1.0
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(stereo)
    wls_filter.setLambda(lamb)
    wls_filter.setSigmaColor(sig)
    
    disparity = stereo.compute(imgLrec, imgRrec)
    disparity2 = stereo2.compute(imgRrec, imgLrec)
    disparity2 = np.int16(disparity2)
    
    filteredImg = wls_filter.filter(disparity, imgL, None, disparity2)
    _, filteredImg = cv2.threshold(filteredImg, 0, max_disparity * 16, cv2.THRESH_TOZERO)
    filteredImg = (filteredImg / 16).astype(np.uint8)
    cv2.imwrite(os.path.join(path, 'dsprty.jpg'), filteredImg)
    
    return filteredImg#, Q

**Disparity image**

<img src="./data/stereo/dsprty.jpg" alt="Drawing" style="width: 1000px;"/> </td>

<div class="alert alert-block alert-warning"><b>QUESTION! </b>  </div>

- **Why is the structure of a Disparity image** _(every pixel is a z)_ **exactly the same as a Digital Elevation Model (DEM)? What does this say about what we have just done?**

<div class="alert alert-block alert-info"><b>BONUS!</b> Change the colour palette from gray-scale. </div> 

{ click in this cell and write your answer here }

#### 1. e. Project to 3D

We are now at the final stage of the stereo-vision process. We want to harvest the values, from the **disparity** map, and convert the _depth_ into 3D information _(a point cloud we can render in a 3D environment)_.

In order to do so we need additional information. We need the distance _(the baseline)_ between the cameras (the eyes).

In [ ]:
baseline = 193.001/2

In [16]:
def Reprojection3D(image, disparity, f, b):
    #- Q. you might need to... 
    Q = np.array([[1, 0, 0, -2964/2], [0, 1, 0, -2000/2],[0, 0, 0, f],[0, 0, -1/b, -124.343/b]])
    
    points = cv2.reprojectImageTo3D(disparity, Q)
    mask = disparity > disparity.min()
    colors = image
    
    out_points = points[mask]
    out_colors = image[mask]
    
    verts = out_points.reshape(-1,3)
    colors = out_colors.reshape(-1,3)
    verts = np.hstack([verts, colors])
    
    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar blue
        property uchar green
        property uchar red
        end_header
        '''
    
    with open(input_dir + '/stereo.ply', 'w') as f:
        f.write(ply_header %dict(vert_num = len(verts)))
        np.savetxt(f, verts, '%f %f %f %d %d %d')
        
def StereoCV():
    #- the function
    img1 = cv2.imread(os.path.join(input_dir, 'im0.png'))
    img2 = cv2.imread(os.path.join(input_dir, 'im1.png'))

    pts1, pts2, matches = PointMatchingAKAZE(img1, img2, path = input_dir)

    #K = findCalibrationMat()
    K = np.array([[3979.911, 0, 1369.115], [0, 3979.911, 1019.507], [0, 0, 1]], dtype = np.float32)
    D = np.zeros((5,1), dtype = np.float32)

    E, R, t = FindEssentialMat(pts1, pts2, matches, K)

    P1 = np.zeros((3,4))
    P1 = np.matmul(K, P1)
    P2 = np.hstack((R, t))
    P2 = np.matmul(K, P2)

    #filteredImg, Q = rectifyToDisparity(img1, img2, K, D, (2964, 2000), R, t, input_dir)
    filteredImg, Q = rectifyToDisparity(img1, img2, K, D, R, t, input_dir)

    f = 3979.911/2
    #Reprojection3D(img1, filteredImg, Q)
    Reprojection3D(img1, filteredImg, f, baseline)

    cv2.destroyAllWindows()

In [17]:
#- execute

StereoCV()

<img src="./data/stereo/result.gif" alt="Drawing" style="width: 1000px;"/> </td>

<div class="alert alert-block alert-warning"><b>TASK! </b>  </div>

- **Capture two photographs of any object and execute this exercise. You will need to determine the distance between the cameras** _(baseline)_ **and create a `calib.txt`.**

   **In no more than 50 words; discuss the result** _(the stereo.ply)_. **You are expected to comment on how the solution can be improved**.

{ click in this cell and write your answer here }

____

## 2. Structure-from-Motion (SfM)

**We will work with the well known [(Christoph Strecha's) Fountain-P11 dataset](https://certis.enpc.fr/demos/stereo/Data/Fountain11/index.html).**

#### 1. c. Fundamental and Essential matrices _(location, scale and rotation)_


In **Photogrammetry** we locate and orient _**one**_ image and are able to calculate a 3D coordinate via the **collinearity equation**. When we do intersect **image rays** from more than one image; we do so from already oriented imagery. 

With **Computer Vision** we **match** corresponding features in photographs. We still resolve for the geometry and pose _(relative orientation)_ of the camera but we do so for _**all cameras simultaneously**_ (more than one at a time). And because our focus is an automated process we also look to harvest 3D data via algorithms. 

The feature matching in **Computer Vision** thus serves two purposes. We recover the _relative orientation_ of all images simultaneously (essential matrix) and describe the relationship between images in the same scene. And use these to map points of one image to lines in another (fundamental matrix). 

Think of an **essential and fundamental matrices** as generalizations of the **collinearity equations** that account for the relative geometry between multiple cameras. They allow for the reconstruction of 3D structures by considering the relationship between corresponding points in multiple cameras.

<div class="alert alert-block alert-warning"><b>TASK / QUESTION! </b>  </div>

- **Execute this NoteBook on a series of photographs you have captured yourself** _(no more than 15)_. **This Notebook must therefore contain the output from you own dataset.**

_images:_

- **pinhole model**: https://kornia.readthedocs.io/en/latest/geometry.camera.pinhole.html
- **disparity**: https://www.e-consystems.com/blog/camera/technology/what-is-a-stereo-vision-camera-2/